# First Steps with GMT/Python


This tutorial will get you started with the basic usage of GMT/Python.
Some of the examples shown here are from the [GMT Tutorial](http://gmt.soest.hawaii.edu/doc/latest/GMT_Tutorial.html#session-one).

## Loading the library

The GMT modules are available as functions and classes in the `gmt` Python package. 
So we'll start by importing it:

In [ ]:
import gmt

## A first map

All figure generation in GMT/Python is handled by the `gmt.Figure` class. 
It has methods to add layers to your figure, like a basemap, coastlines, and data.

We start a new figure by creating an instance of `gmt.Figure`:

In [ ]:
fig = gmt.Figure()

We add elements to the figure using its methods. For example, lets add the coastlines of Central America to a 6 inch wide map using the Mercator projection (`M`). Our figure will also have a nice frame with automatic ticks.

In [ ]:
fig.coast(region=[-90, -70, 0, 20], projection='M6i', land='chocolate', 
          frame=True)

You can see a preview of the figure directly in the [Jupyter notebook](http://jupyter.org) using `fig.show()`.

In [ ]:
fig.show()

To open a PDF preview of the figure using your default PDF viewer use:

```python
fig.show(external=True)
```

This is useful when using the Python shell or IPython terminal app. 
However, **this command will not interrupt your Python process**. 
So using it in a Python script will not work because the script will finish and delete the generated previews.
Use `fig.savefig` to save the figure to a file instead (see below).

## A note for experienced GMT users

You'll probably have noticed several things that are different from classic command-line GMT.
Many of these changes reflect the new GMT [modern execution mode](http://gmt.soest.hawaii.edu/projects/gmt/wiki/Modernization) that will be part of the future 6.0 release.
A few are GMT/Python exclusive (like the long argument names).

1. The name of method is `coast` instead of `pscoast`. As a general rule, all `ps*` modules had their `ps` removed. The exceptions are: `psxy == plot`, `psxyz == plot3d`, and `psscale == colorbar`.
2. The arguments don't use the GMT 1-letter syntax (R, J, B, etc). Those are still available as aliases and the methods will accept them (see below). 
3. Arguments like `region` can take lists instead of strings like `1/2/3/4`. You can still use the string form but the list form is easier in Python.
4. If a GMT argument has no options (like `-B` instead of `-Baf`), use a `True` value instead. An empty string would also be acceptable.
5. There is no output redirecting to a PostScript file. The figure is generated in the background and will only be shown or saved when you ask for it.

We could have generated the figure above using the classic GMT argument names (but not the module names):

In [ ]:
fig_alias = gmt.Figure()
fig_alias.coast(R='-90/-70/0/20', J='M6i', G='gray', S="blue", B=True)
fig_alias.show()

## Saving the figure

Unlike the GMT command-line interface, **no figure file was generated until you ask for one**.  
That means that `fig.show` won't produce a figure file.

Use method `fig.savefig` (based on the [matplotlib](https://matplotlib.org/) function) to save your figure:

In [ ]:
fig.savefig('first-steps-central-america.png')

If you're running a Python script, you can tell `fig.savefig` to open the figure in an external viewer:

```python
fig.savefig('first-steps-central-america.png', show=True)
```

## Plotting point data

We can use `gmt.Figure.plot` to plot data on our map.

First, lets create some fake data to plot using numpy:

In [ ]:
import numpy as np

In [ ]:
# See the random number generator so we always 
# get the same numbers
np.random.seed(42)

ndata = 100
region = [150, 240, -10, 60]

# Create fake epicenter, magnitude, and depth values
lon = np.random.uniform(region[0], region[1], ndata)
lat = np.random.uniform(region[2], region[3], ndata)
magnitude = np.random.uniform(1, 9, ndata)
depth = np.random.uniform(1, 600, ndata)

Now we can plot the data using `Figure.plot` and passing the coordinate arrays as the x and y arguments:

In [ ]:
fig = gmt.Figure()
# Plot using a global region ('g') and an Othographic projection ('G')
fig.coast(region='g', projection='G200/20/6i', frame='ag', land='grey', 
          water='white')
# Plot using circles (c) of 0.1 cm
fig.plot(x=lon, y=lat, style='c0.1c', color='black')
fig.show()

We can make the size of the markers follow the magnitude values by passing in the argument `sizes` to `Figure.plot`. We'll need to scale the magnitude (the size is in centimeters) and use a power law to actually see all the different sizes.

In [ ]:
fig = gmt.Figure()
fig.coast(region='g', projection='G200/20/6i', frame='g', 
          land='grey', water='white')
fig.plot(x=lon, y=lat, sizes=0.01*(1.7**magnitude), style='cc', 
         color='black')
fig.show()

We can also map the colors of the markers to the depths by passing an array to the `color` argument annd providing a colormap name (`cmap`). We can even use the new matplotlib colormap `"viridis"`. In order to highlight the depth range, we'll use the log10 of the depth instead.

In [ ]:
log_depth = np.log10(depth)
# We need to normalize the color because 'plot' can't 
# autoscale the cmap to our data.
log_depth_norm = log_depth/log_depth.max()

In [ ]:
fig = gmt.Figure()
fig.coast(region='g', projection='G200/20/6i', frame='g', 
          land='grey', water='white')
fig.plot(x=lon, y=lat, sizes=0.01*(1.7**magnitude), style='cc', 
         color=log_depth_norm, cmap='viridis')
fig.show()

### Plotting directly from a file

Sometimes you'll have data in a file that you just want to plot without having to load it into Python. You can use the `data` argument to specify the file name instead `x` and `y`. GMT will take care of loading your data and plotting.

In [ ]:
# Save our fake data to a file.
np.savetxt('first-steps-data.txt', 
           np.transpose([lon, lat, magnitude, depth]))

We can use the `columns` argument to control which columns we want to plot as x, y, color, and size, respectively. Adding `l` to a column will take the log10 of it and `sVALUE` will multiply it by that value. There are some limitations to this method on the kind of operations we can do with the data before plotting. For example, we can't use a power law for the magnitude.

In [ ]:
fig = gmt.Figure()
# Make a Mercator map this time.
fig.coast(region=region, projection='M6i', frame='afg',
          land='white', water='skyblue')
fig.plot(data='first-steps-data.txt', style='cc', cmap='inferno', 
         columns=[0, 1, '3ls0.36', '2s0.05'])
fig.show()

### Using sample data from GMT

GMT ships some sample data that can be accessed by passing `@data_file` as the `data` argument. For example, we can plot the earthquake epicenters from the `tut_quakes.ngdc` sample dataset:

In [ ]:
fig = gmt.Figure()
fig.coast(region=[130, 150, 35, 50], projection='M6i', 
          frame='afg', shorelines=True, land='gray', 
          water='lightblue')
fig.plot(data='@tut_quakes.ngdc', style='c0.3c', 
         color='blue', columns=[4, 3])
fig.show()